### Importing the Libaries

In [1]:
import numpy as np
import pandas as pd
from pulp import *
import math
import itertools
import pymorton

### Defining the Data

In [2]:
def define_data():
    df = {}
    
    df['weight'] = [200] *2 + [80] * 20 + [50] * 15 + [30] * 12 + [25] * 20
    df['boxes'] = [i for i in range(len(df['weight']))]
    
    df['length'] = [30] *2 + [15] * 20 + [10] * 15 + [8] * 12 + [5] * 20
    df['breadth'] = [25]*2 + [15] * 20 + [10] * 15 + [8] * 12 + [5] * 20
    df['height'] = [25] * 2 + [15] * 20 + [10] * 15 + [8] * 12 + [5] * 20
    df['volume'] = list(np.array(df['length']) * np.array(df['breadth']) * np.array(df['height']))
#     df['truck_weight'] = 1000
    df['total_truck_fleet'] = [i for i in range(len(df['weight']))]
    return df

In [3]:
data = define_data()
pd.DataFrame(data).head()

,weight,boxes,length,breadth,height,volume,total_truck_fleet
0,200,0,30,25,25,18750,0
1,200,1,30,25,25,18750,1
2,80,2,15,15,15,3375,2
3,80,3,15,15,15,3375,3
4,80,4,15,15,15,3375,4


In [158]:
df = pd.DataFrame()
df['weight'] =[150, 150]
df['boxes'] = [0, 1]

df['length'] = [200, 200]
df['breadth'] = [150, 150]
df['height'] = [60, 55]
df['volume'] = list(np.array(df['length']) * np.array(df['breadth']) * np.array(df['height']))
#     df['truck_weight'] = 1000
df['total_truck_fleet'] = [i for i in range(len(df['weight']))]

In [159]:
data = df.copy()
data

,weight,boxes,length,breadth,height,volume,total_truck_fleet
0,150,0,200,150,60,1800000,0
1,150,1,200,150,55,1650000,1


In [160]:
class OptimiseBoxLoading:
    def __init__(self, box_info_path, truck_info_path):
        self.df_boxes = pd.read_csv(box_info_path)
        self.df_truck = pd.read_csv(truck_info_path)
        
    def define_truck_info(self):
        length_of_truck = self.df_truck['truck_length'].iloc[0]
        width_of_truck = self.df_truck['truck_width'].iloc[0]
        height_of_truck = self.df_truck['truck_height'].iloc[0]
        volume_of_truck = self.df_truck['truck_capacity'].iloc[0]
        weight_of_truck = self.df_truck['truck_weight_limit'].iloc[0]

        min_height = self.df_boxes['H'].min()
        min_width = self.df_boxes['W'].min()
        min_length = self.df_boxes['L'].min()
        
        x_axis = math.ceil(length_of_truck / min_length)
        y_axis = math.ceil(breadth_of_truck / min_breadth)
        z_axis = math.ceil(height_of_truck / min_height)
        
        self.total_truck_fleet = self.df_boxes.shape[0]
        
        self.min_height = min_height
        self.min_width = min_width
        self.min_length = min_length
        
        self.volume_of_truck = volume_of_truck
        self.weight_of_truck = weight_of_truck
        
        self.x_axis = x_axis
        self.y_axis = y_axis
        self.z_axis = z_axis
        
        return self

        
    
    def define_placement_dict(self):
        dict_truck_info = {}
        for tr in self.total_truck_fleet:
            for i in range(self.x_axis):
                for j in range(self.y_axis):
                    for k in range(self.z_axis):
                        dict_truck_info[(tr, i, j ,k)] = {'box_loaded': LpVariable("loaded", 0, 1, cat = 'Binary'),
                                                         'length_loaded': LpVariable("length", 0, self.length_of_truck, cat = 'Continuous'),
                                                         'breadth_loaded': LpVariable("breadth", 0, self.breadth_of_truck, cat = 'Continuous'),
                                                         'height_loaded': LpVariable("height", 0, self.height_of_truck, cat = 'Continuous'),
                                                         'volume_loaded': LpVariable("volume", 0, self.volume_of_truck, cat = 'Continuous'),
                                                         'surface_area': LpVariable(f"{tr}_surface_area_{(i, j, k)}", 0, 
                                                                                    self.length_of_truck * self.height_of_truck, 
                                                                                    cat = 'Continuous')}
        return dict_truck_info
    
    def dict_update(self, dict_truck_info, truck_num, boxes_data, box_num, axes, dimensions, update_dimensions):
        x_ax = axes[0]
        y_ax = axes[1]
        z_ax = axes[2]

        remaining_length = dimensions[0]
        remaining_breadth = dimensions[1]
        remaining_height = dimensions[2]

        dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['box_loaded'] = 1
        dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['length_loaded'] = data['length'][box_num]
        dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['breadth_loaded'] = data['breadth'][box_num]
        dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['heigth_loaded'] = data['height'][box_num]

        dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['surface_area'] = data['height'][box_num] * data['length'][box_num]

        if update_dimensions[0] is True:
            remaining_length -= dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['length_loaded']

        if update_dimensions[1] is True:
            remaining_breadth -= dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['breadth_loaded']

        if update_dimensions[2] is True:
            remaining_height -= dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['height_loaded']

        return dict_truck_info, remaining_length, remaining_breadth, remaining_height
    
    def build_function(self):
        
        prob = LpProblem("truckLoading", LpMinimize)
        
        dict_truck_info = self.define_placement_dict()
        
        list_of_axes = [[x for x in range(self.x_axis)] , [y for y in range(self.y_axis)] , [z for z in range(self.gz_axis)]]
        list_coordinates = list(itertools.product(*list_of_axes))


        remaining_weight = self.weight_of_truck# - data['weight'][i]
        remaining_volume = self.volume_of_truck # - data['vlume'][i]
        
        
        boxes_loaded = LpVariable.dicts("AtLocation",
        [(i, j) for i in range(self.df_boxes.shape[0])
        for j in self.total_truck_fleet],
        0, 1, LpBinary)

        for j in self.total_truck_fleet:
            remaining_length = self.length_of_truck
            remaining_breadth = self.breadth_of_truck
            remaining_height = self.height_of_truck
            remaining_weight = self.weight_of_truck
            remaining_volume = self.volume_of_truck

            for i in range(self.df_boxes.shaoe[0]):
        #         if boxes_loaded[(i, j)].varValue:
        #             if boxes_loaded[(i, j)].varValue > 0:
                for val in list_coordinates:
                    x_ax = val[0]; y_ax = val[1]; z_ax = val[2]

                while(remaining_volume >0 and remaining_weight > 0):
                    if x_ax >= x_axis and y_ax >= y_axis and z_ax >= z_axis:
                        boxes_loaded[(i, j)].varValue = 0
                        break
                    elif x_ax >= x_axis and y_ax >= y_axis and z_ax < z_axis:
                        x_ax = val[0]; y_ax = val[1]; z_ax += 1
                        continue
                    elif x_ax >= x_axis and y_ax < y_axis and z_ax < z_axis:
                        x_ax = val[0]; y_ax += 1
                        continue

                    elif y_ax >= y_axis:
                        x_ax = val[0]; y_ax = val[1]; z_ax += 1
                        continue

                    else:
                        boxes_loaded[(i, j)].varValue = 0
                        break


                    if dict_truck_info[(j, x_ax, y_ax, z_ax)]['box_loaded'] == 1:
                        if remaining_length >0 and remaining_breadth > 0 and remaining_height > 0:
                            x_ax += 1
                            continue
                        elif remaining_length <= 0 and remaining_breadth > 0 and reamining_height > 0:
                            x_ax = val[0]; y_ax += 1
                            continue
                        elif remaining_length <= 0 and remaining_breadth <= 0 and reamining_height > 0:
                            x_ax = val[0]; y_ax = val[1]; z_ax += 1
                            continue
                        else:
                            break


                    elif dict_truck_info[(j, x_ax, y_ax, z_ax)]['box_loaded'] == 0:
                        if remaining_length >0 and remaining_breadth > 0 and reamining_height > 0:
                            if z_ax == 0:
                                dict_truck_info,\
                                remaining_length,\
                                remaining_breadth,\
                                remaining_height = self.dict_update(dict_truck_info = dict_truck_info,
                                                             truck_num = j,
                                                             boxes_data = data,
                                                             box_num = i,
                                                             axes = [a_ax, y_ax, z_ax], 
                                                             dimensions = [remaining_length, remaining_breadth, remaining_height],
                                                             update_dimensions = [False, True, False])
                                boxes_loaded[(i, j)].varValue = 1
                            else:
                                if dict_truck_info[(j, x_ax, y_ax, z_ax-1)]['volume_loaded'] > data['volume'][i]:
                                    dict_truck_info,\
                                    remaining_length,\
                                    remaining_breadth,\
                                    remaining_height = self.dict_update(dict_truck_info = dict_truck_info,
                                                                 truck_num = j,
                                                                 boxes_data = data,
                                                                 box_num = i,
                                                                 axes = [a_ax, y_ax, z_ax], 
                                                                 dimensions = [remaining_length, remaining_breadth, remaining_height],
                                                                 update_dimensions = [False, True, False])
                                    boxes_loaded[(i, j)].varValue = 1
                                else:
                                    x_ax += 1
                                    continue

                        elif remaining_length <= 0 and remaining_breadth > 0 and reamining_height > 0:
                            x_ax = val[0]; y_ax += 1
                            continue
                        elif remaining_length <= 0 and remaining_breadth <= 0 and reamining_height > 0:
                            x_ax = val[0]; y_ax = val[1]; z_ax += 1
                            continue
                        else:
                            boxes_loaded[(i, j)].varValue = 0
                            break
        
        y = [LpVariable("t{0}".format(i+1), cat=LpBinary) for i in data['total_truck_fleet']]
        
        
        prob += lpSum([y[i] for i in data['total_truck_fleet']])
     
        #For each box, it can be placed in only one truck
        for i in data['boxes']:
            prob += lpSum(boxes_loaded[(i, j)] for j in data['total_truck_fleet']) == 1

        #for each truck,
        #    summation of weight of all boxes should be less than total weight truck can carry
        #    summation of volume of all boxes should be less than volume of the truck
        for j in data['total_truck_fleet']:
            prob += lpSum(boxes_loaded[(i, j)] * data['weight'][i] for i in data['boxes']) <= weight_of_truck * y[j]
            prob += lpSum(boxes_loaded[(i, j)] * data['volume'][i] for i in data['boxes']) <= volume_of_truck * y[j]

        for truck in data['total_truck_fleet']:
            for y_ in range(1, y_ax):
                prob += lpSum([dict_truck_info[truck, x_, y_, z_]['surface_area'] for x_ in range(1, x_ax) for z_ in range(1, z_ax)]) <= breadth_of_truck * height_of_truck
    
    
    def generate_results(self):
        prob.solve()
        TOL = 0.00001
        truck_num = 0
        total_trucks_used = 0
        for j in data['total_truck_fleet']:
            if y[j].varValue > TOL:
                total_trucks_used += 1
        print(f"Total Trucks Used: {total_trucks_used}")
        print("\n")
        print("="* 100)
        for j in data['total_truck_fleet']:

            if y[j].varValue > TOL:
                print(f"Truck Number: {truck_num+1}")
                print("=" * 100)
                total_weight_loaded = 0
                total_num_boxes = 0
                total_volume_loaded = 0
                for i in data['boxes']:
                    if boxes_loaded[(i, j)].varValue > TOL:
                        total_num_boxes += 1
                        total_weight_loaded += data['weight'][i]
                        total_volume_loaded += data['volume'][i]
                print(f'Total Boxes Loaded: {total_num_boxes}')
                print(f"Total Weight Loaded: {total_weight_loaded}")
                print(f"Total Volume Loaded: {total_volume_loaded}")
                print("-" * 80)

                for i in data['boxes']:
                    if boxes_loaded[(i, j)].varValue > TOL:

                        box_num = data['boxes'][i]
                        len_ = data['length'][i]
                        br_ = data['breadth'][i]
                        hei_ = data['height'][i]
                        wt_ = data['weight'][i]
                        print(f"\t Box Number: {box_num}", f"\t Dimensions of Box: {len_, br_, hei_}", f"\t Weight of Box: {wt_}", sep = "\n")
                        print("-" * 80)
        #         print ("Truck ", j, " holds ", \
        #         [i for i in data['boxes']
        #         if boxes_loaded[(i, j)].varValue > TOL])
                print("=" * 100)
                truck_num+= 1

## New Approach

#### Define Data

In [161]:
data = pd.DataFrame(data)
data = data.head(7)
data.head()

,weight,boxes,length,breadth,height,volume,total_truck_fleet
0,150,0,200,150,60,1800000,0
1,150,1,200,150,55,1650000,1


In [162]:
data

,weight,boxes,length,breadth,height,volume,total_truck_fleet
0,150,0,200,150,60,1800000,0
1,150,1,200,150,55,1650000,1


### Define the Problem

In [163]:
problem = LpProblem("truck_loading", LpMinimize)

### Define the Objective 

In [164]:
y = [LpVariable("t{0}".format(i+1), cat=LpBinary) for i in data['total_truck_fleet']]

In [165]:
problem += lpSum([y[i] for i in data['total_truck_fleet']])

### Define the Variables

##### Define the truck dims and the axes

In [166]:
length_of_truck = 200
breadth_of_truck = 150
height_of_truck = 120

min_height = min(data['height'].values)
min_breadth = min(data['breadth'].values)
min_length = min(data['length'].values)
volume_of_truck = length_of_truck * breadth_of_truck * height_of_truck
weight_of_truck = 800
x_axis = math.ceil(length_of_truck / min_length)
y_axis = math.ceil(breadth_of_truck / min_breadth)
z_axis = math.ceil(height_of_truck / min_height)

In [167]:
max_weight_for_each_stack = 350

In [168]:
list_of_axes = [[x for x in range(x_axis)] , [y for y in range(y_axis)] , [z for z in range(z_axis)]]
list_coordinates = list(itertools.product(*list_of_axes))

##### Dictionary Variable

In [169]:
post_box_loaded = LpVariable.dicts("At_truck_location",
                                   [(i,j, k) for i in data['boxes']
                                    for j in data['total_truck_fleet']
                                    for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]],
                                   0, 1, LpBinary)

In [170]:
len(post_box_loaded)

12

##### Adding Constraints

In [171]:
#For each box, it can be placed in only one truck

for i in data['boxes']:
    problem += lpSum(post_box_loaded[(i, j, k)] for j in data['total_truck_fleet']
                 for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) == 1

In [172]:
#for each truck,
#    summation of weight of all boxes should be less than total weight truck can carry
#    summation of volume of all boxes should be less than volume of the truck


for j in data['total_truck_fleet']:
    problem += lpSum(post_box_loaded[(i, j, k)] * data['weight'][i] for i in data['boxes'] 
                    for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) <= weight_of_truck * y[j]
    problem += lpSum(post_box_loaded[(i, j, k)] * data['volume'][i] for i in data['boxes'] 
                    for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) <= volume_of_truck * y[j]


In [173]:
# # for each truck,
# #     for each box,
# #         it can be placed in only one position


# for j in data['total_truck_fleet']:
#     for i in data['boxes']:
#         problem += lpSum(post_box_loaded[(i, j, k)] for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]) == 1

In [174]:
truck 1:
    y --- 0 to 2:
        x --- 0 to 2:
            for z --- 0 to 2:
                sum(box_loaded in i,j ,k * weight of box loaded)

SyntaxError: invalid syntax (<ipython-input-174-6efa71e514da>, line 1)

In [175]:
# for each truck,
#     for each x,y point
#         1. sum of all boxes placed along the z axis on that point should be less than height of truck
#         2. weight of all boxes along a stack must be less than permissible weight
#         3. surface area of a box must be less than or equal to that of the box placed below it


for j in data['total_truck_fleet']:
    for y_ in range(y_axis):
        for x_ in range(x_axis):
            problem += lpSum(post_box_loaded[(i, j, k)] * data['height'].iloc[i] for i in data['boxes'] 
                    for k in [pymorton.interleave(x_, y_, z_) for z_ in range(z_axis)]) <= height_of_truck * y[j]
            
            problem += lpSum(post_box_loaded[(i, j, k)] * data['weight'].iloc[i] for i in data['boxes'] 
                    for k in [pymorton.interleave(x_, y_, z_) for z_ in range(z_axis)]) <= max_weight_for_each_stack * y[j]
            
# for j in data['total_truck_fleet']:
#     for y_ in range(y_axis):
#         for x_ in range(x_axis):
#             problem += lpSum(post_box_loaded[(i, j, k)] * data['length'].iloc[i] * data['breadth'].iloc[i]
#                         for i in data['boxes'] 
#                         for k in [pymorton.interleave(x_, y_, z_) for z_ in range(1, z_axis)]) <= lpSum(
                        
#                         post_box_loaded[(i, j, k-1)] * data['length'].iloc[i] * data['breadth'].iloc[i]
#                         for i in data['boxes'] 
#                         for k in [pymorton.interleave(x_, y_, z_) for z_ in range(1, z_axis)])

In [176]:
# for each truck,
#     for each x,z point
#         sum of all boxes placed along the y axis on that point should be less than width of truck


for j in data['total_truck_fleet']:
    for z_ in range(z_axis):
        for x_ in range(x_axis):
#             print([k for k in [pymorton.interleave(x, y, z_) for z_ in range(z_axis)]])
            problem += lpSum(post_box_loaded[(i, j, k)] * data['breadth'].iloc[i] for i in data['boxes'] 
                    for k in [pymorton.interleave(x_, y_, z_) for y_ in range(y_axis)]) <= breadth_of_truck * y[j]

In [177]:
# for each truck,
#     for each y,z point
#         sum of all boxes placed along the x axis on that point should be less than length of truck


for j in data['total_truck_fleet']:
    for z_ in range(z_axis):
        for y_ in range(y_axis):
#             print([k for k in [pymorton.interleave(x, y, z_) for z_ in range(z_axis)]])
            problem += lpSum(post_box_loaded[(i, j, k)] * data['length'].iloc[i] for i in data['boxes'] 
                    for k in [pymorton.interleave(x_, y_, z_) for x_ in range(x_axis)]) <= length_of_truck * y[j]

In [178]:
# for j in data['total_truck_fleet']:
#     for y_ in range(y_axis):
#         for x_ in range(x_axis):
#             for z_ in range(1, z_axis):
#                 for i in range(data['boxes'].shape[0]):
#                     k = pymorton.interleave(x_, y_, z_)

#                     problem += (post_box_loaded[(i, j, k)] * data['length'].iloc[i] * data['breadth'].iloc[i]) <= (
#                         post_box_loaded[(i, j, k-1)] * data['length'].iloc[i] * data['breadth'].iloc[i])

In [179]:
x_

0

In [180]:
problem

truck_loading:
MINIMIZE
1*t1 + 1*t2 + 0
SUBJECT TO
_C1: At_truck_location_(0,_0,_0) + At_truck_location_(0,_0,_32)
 + At_truck_location_(0,_0,_4) + At_truck_location_(0,_1,_0)
 + At_truck_location_(0,_1,_32) + At_truck_location_(0,_1,_4) = 1

_C2: At_truck_location_(1,_0,_0) + At_truck_location_(1,_0,_32)
 + At_truck_location_(1,_0,_4) + At_truck_location_(1,_1,_0)
 + At_truck_location_(1,_1,_32) + At_truck_location_(1,_1,_4) = 1

_C3: 150 At_truck_location_(0,_0,_0) + 150 At_truck_location_(0,_0,_32)
 + 150 At_truck_location_(0,_0,_4) + 150 At_truck_location_(1,_0,_0)
 + 150 At_truck_location_(1,_0,_32) + 150 At_truck_location_(1,_0,_4) - 800 t1
 <= 0

_C4: 1800000 At_truck_location_(0,_0,_0)
 + 1800000 At_truck_location_(0,_0,_32) + 1800000 At_truck_location_(0,_0,_4)
 + 1650000 At_truck_location_(1,_0,_0) + 1650000 At_truck_location_(1,_0,_32)
 + 1650000 At_truck_location_(1,_0,_4) - 3600000 t1 <= 0

_C5: 150 At_truck_location_(0,_1,_0) + 150 At_truck_location_(0,_1,_32)
 + 150 At_t

In [181]:
problem.solve()

1

In [182]:
TOL = 0.00001
truck_num = 0
total_trucks_used = 0
for j in data['total_truck_fleet']:
    if y[j].varValue > TOL:
        total_trucks_used += 1
print(f"Total Trucks Used: {total_trucks_used}")

Total Trucks Used: 1


In [183]:
for j in data['total_truck_fleet']:
    for i in data['boxes']:
        for k in [pymorton.interleave(i, j, k) for (i, j, k) in list_coordinates]:
            if post_box_loaded[(i, j, k)].varValue > TOL:
                print(f"box_{i}_loaded_in_truck_{j}")

box_0_loaded_in_truck_0
box_1_loaded_in_truck_0
